<font face="Calibri" size="2"> <i>SBAE - Notebook Series - Part 2, version 0.1,  June 2022. Andreas Vollrath, UN-Food and Agricultural Organization, Rome</i>
</font>

![title](images/header.png)

# II - SBAE Time-Series Extraction & Change Detection
### Extract various time-series outputs for point data from Google Earth Engine
-------

This notebook takes you through the process of extracting outputs from various time-series change detection algorithms and structure them in a so-called data-frame (e.g. tabular structure).

### 1 - Import libs

**ONLY EXECUTE THIS CELL**

In [1]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
from sepal_ui.mapping import SepalMap
# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

Styles()

/home/vollrath/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


### 2 - Basic Input Variables

**FILL IN YOUR INPUTS**

In [2]:
# Area of Interest

# country/province case example
country = 'Buhweju'   
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME', country)) # here any feature collection can be selected

# Asset case
#aoi = ee.FeatureCollection("my_aoi_feature_collection")

# Point Feature Collection from where to extract points
fc = ee.FeatureCollection("users/andreasvollrath/01_test_sbae_points").filterBounds(aoi)

# the column of a unique point identifier in your dataset
point_id_name = 'point_id'

grid_size = 0.25 # that's the size of the grid we are parallelizing on in degrees
workers = 10 # number of parallel EE requests

#### 2b - Check inputs visually

In [3]:
# create the grid
grid, grid_fc = h.processing_grid(aoi, grid_size, grid_size)

Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

# add layers
Map.addLayer(grid_fc)
Map.addLayer(fc)
Map.addLayer(aoi)
Map

SepalMap(center=[-0.3203418090924264, 30.347812392179087], controls=(ZoomControl(options=['position', 'zoom_in…

#### 2c - Time-series parameter settings

In [4]:
# start of calibration period (mainly for bfast)
start_calibration = "2014-01-01"  # YYYY-MM-DD format

# Actual period of interest, i.e. monitoring period
start_monitor = "2017-01-01"  # YYYY-MM-DD format
end_monitor =  "2022-01-01"  # YYYY-MM-DD format

# Directory where output and temp files will go
outdir = 'results_new'  # goes to module_results/sbae_point_analysis if left to None

# Select algorithms to run
cusum_deforest = True
bfast_monitor = True
bs_slope = True
ts_metrics = True
ccdc = True
landtrendr = True
global_products = True

# select the bands to extract
bands=['green', 'red', 'nir', 'swir1', 'swir2', 'ndfi'] # other choices: ndfi, ndmi, mndwi, brightness, greenness, wetness

# select the band for univariate ts-analysis (has to be inside bands list)
ts_band = 'ndfi'

# select the resolution to which the satellite data will be resized.
scale=100  # in meters

### DO NOT CHANGE YET ###
satellite='Landsat'
max_cloud_cover = 75  # in percentage (0-100)

### 3- Algorithm parameter settings

**Edit for advanced users, otherwise just execute**

In [5]:
# bfast parameters
bfast_params = {
    'run': bfast_monitor,
    'start_monitor': start_monitor, 
    'freq': 365,
    'k': 3, 
    'hfrac':0.25, 
    'trend': True, 
    'level':0.05, 
    'backend':'python'
}

# cusum parameters
cusum_params = {
    'run': cusum_deforest,
    'nr_of_bootstraps': 1000
}

# slope parameters
bs_slope_params = {
    'run': bs_slope,
    'nr_of_bootstraps': 1000
}

# time-series metrics
ts_metrics_params = {
    'run': ts_metrics,
    'outlier_removal': False,
    'z_threshhold': 3
}

# ccdc parameters
ccdc_params = {
    'run': ccdc,
    'breakpointBands': ['green','red','nir','swir1','swir2'],
    'tmaskBands': ['green','swir2'],
    'minObservations': 6,
    'chiSquareProbability': .99,
    'minNumOfYearsScaler': 1,
    'dateFormat': 2,
    'lambda': 20,
    'maxIterations': 1000
}


landtrendr_params = { 
        'run': landtrendr,
        'maxSegments':            6,
        'spikeThreshold':         0.9,
        'vertexCountOvershoot':   3,
        'preventOneYearRecovery': True,
        'recoveryThreshold':      0.25,
        'pvalThreshold':          0.05,
        'bestModelProportion':    0.75,
        'minObservationsNeeded':  3
}

# global products parameters
global_products = {
    'run': global_products,
    'gfc': True,  # will include tree-cover 2000, loss, gain, lossyear
    'tmf': True,  # will include deforestation and degradation year for tropical moist forests
    'tmf_years': True, # will include classes per year - according to the monitor period
    'esa_lc20': True, # will include ESA LandCover Product class
    'copernicus_lc': True, # will include ESA LandCover Product class - acording to the monitoring years
    'esri_lc': True,  # will include the classes from ESRI World Cover 2020
    'lang_tree_height': True, # returns the Tree Height from Lang et al 2022
    'potapov_tree_height': True, # returns the tree height from Potapov et al. 2019 
    'dynamic_world_tree_prob': True, # returns Min, Max, Mean and StdDev of the trees probability for the monitoring period
    'dynamic_world_class_mode': True, # returns the mode of the class for the monitoring period
    'elevation': True  # returns elevation, slope and aspect
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir': outdir,
    'workers': workers,
    'ts_params': {
        'start_calibration': start_calibration,
        'start_monitor': start_monitor,
        'end_monitor': end_monitor,
        'point_id': point_id_name,
        'grid_size': grid_size,
        'bands': bands,
        'ts_band': ts_band,
        'satellite': satellite,
        'scale': scale,
        'max_cc': max_cloud_cover,
        'outlier_removal': True,
        'smooth_ts': True
    },
    'bfast_params': bfast_params,
    'cusum_params': cusum_params,
    'bs_slope_params': bs_slope_params,
    'ts_metrics_params': ts_metrics_params,
    'ccdc_params': ccdc_params,
    'landtrendr_params': landtrendr_params,
    'global_products': global_products
}

### 4 - Run the time-series data extraction

**Execute only**

In [6]:
h.get_change_data(aoi, fc, config_dict)

 Parallelizing time-series extraction on 10 threads for a total of 4 grid cells.
 Processing gridcell 1
 Processing gridcell 3
 Processing gridcell 2
 Grid cell 0 does not contain any points. Going on with next grid cell.


2022-07-22 14:41:13.006856: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-22 14:41:13.006913: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (4a44e7f5084d): /proc/driver/nvidia/version does not exist
2022-07-22 14:41:13.007833: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


None
 Grid cell 3 with 4 points done in: 0:01:19.419646
None
 Grid cell 2 with 1 points done in: 0:01:46.233039
None
 Grid cell 1 with 22 points done in: 0:02:40.016439


/home/vollrath/.local/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


 Deleting temporary files
 Processing has been finished successfully. Check for final_results files in your output directory.


In [7]:
import pandas as pd
pd.read_pickle('results_new/results_Landsat_ndfi_2014-01-01_2017-01-01_2022-01-01_0.25.pickle')#['bfast_magnitude']

,LON,LAT,PLOTID,aspect,dw_class_mode,dw_tree_prob__max,dw_tree_prob__min,dw_tree_prob__stdDev,dw_tree_prob_mean,elevation,...,cusum_confidence,cusum_magnitude,ts_mean,ts_sd,ts_min,ts_max,bs_slope_mean,bs_slope_sd,bs_slope_max,bs_slope_min
0,30.468090,-0.396305,33,161,6,50,3,9,14,1525.170044,...,0.480377,19381.306641,6580.087853,1160.748748,3938.333333,8834.500000,244.822725,82.385746,488.975876,-75.720635
1,30.468090,-0.357111,34,96,2,45,2,7,9,1789.069946,...,0.317560,33306.710938,1345.785278,2559.416866,-4375.400000,7575.000000,795.342042,173.151166,1299.752379,237.010767
2,30.468090,-0.317917,35,85,1,54,7,11,28,1635.560059,...,0.452518,25522.580078,7349.909000,1788.577751,2464.000000,10000.000000,593.633805,161.339816,1066.141027,22.357924
3,30.468090,-0.278723,36,68,2,15,2,2,5,1946.380005,...,0.499020,48357.531250,2502.917656,2674.245835,-5372.666667,7636.500000,989.860702,163.728863,1496.925055,513.353893
4,30.208986,-0.317917,2,82,1,75,73,0,74,1395.810059,...,0.366263,3206.301758,9665.698076,208.639962,8912.000000,10000.000000,41.302945,10.819719,77.920017,4.565494
5,30.208986,-0.278723,3,45,1,75,72,0,74,1442.760010,...,0.203171,2638.458008,9549.396284,225.585180,8636.000000,10000.000000,27.229358,13.737422,65.929849,-12.589615
6,30.260807,-0.396305,6,230,1,67,6,14,32,1629.290039,...,0.556636,42051.593750,6143.145890,1906.199010,1298.333333,8703.666667,-127.481451,120.830798,228.880080,-587.081615
7,30.260807,-0.357111,7,125,1,76,10,17,67,1464.530029,...,0.611946,53130.203125,8856.064615,2255.537310,2334.250000,10000.000000,974.639191,139.071443,1376.338051,441.799000
8,30.260807,-0.317917,8,36,1,74,71,0,72,1687.920044,...,0.001124,798.015625,9929.712931,94.704744,9545.750000,10000.000000,0.367568,6.196670,21.022823,-20.417532
9,30.260807,-0.278723,9,24,1,74,4,26,35,1702.689941,...,0.211330,20091.396484,8070.788384,1669.017441,3227.000000,10000.000000,-310.715446,91.392181,-35.061628,-641.773125


In [8]:
params= {'start_monitor': '2010-01-01'}

params.update(start_monitor=dt.strptime(bfast_params['start_monitor'], '%Y-%m-%d'))
params

{'start_monitor': datetime.datetime(2017, 1, 1, 0, 0)}